<h1><b>Will You Survive On Titanic AI</b></h1>
<h2>Importing Libraries and Database</h2>

In [1]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns

In [2]:
titanic = pd.read_csv(r"titanic.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
# sns.pairplot(titanic, hue="Survived")

<h1>Data Cleanup</h1>

In [4]:
titanic = titanic.drop(columns=['Name', 'Cabin','Embarked','Ticket'])
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,0,3,male,34.5,0,0,7.8292
1,893,1,3,female,47.0,1,0,7.0000
2,894,0,2,male,62.0,0,0,9.6875
3,895,0,3,male,27.0,0,0,8.6625
4,896,1,3,female,22.0,1,1,12.2875


In [5]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row,column]-min1)/(max1-min1)
                x.at[row,column]=xprim 
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [6]:
DataProcessing.shuffle(titanic)

for row in range(len(titanic)-1,-1,-1):
    if titanic.at[row,'Sex']=="male":
        titanic.at[row,'Sex']=1
    elif titanic.at[row,'Sex']=="female":
        titanic.at[row,'Sex']=0
    else:
        titanic = titanic.drop(row,axis=0)

        
# values=titanic.select_dtypes(exclude="object")
# columnNames=values.columns.tolist()
# for column in columnNames:
# column = "PassengerId"
# for row in range(len(titanic)-1,-1,-1):
#     print(titanic.at[row,column])
#     temp = titanic.at[row,column]
#     if isinstance(temp, (int, float, complex)):
#         print(titanic.at[row,column])
        
        
#     else:
#         print(titanic.iloc[row])
#         titanic = titanic.drop(row,axis=0)


titanic.head()


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1177,0,3,1,36.0,0,0,7.2500
1,1150,1,2,0,19.0,0,0,13.0000
2,1171,0,2,1,22.0,0,0,10.5000
3,963,0,3,1,21.0,0,0,7.8958
4,1289,1,1,0,48.0,1,1,79.2000


In [7]:
DataProcessing.normalization(titanic)
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.683453,0,1.0,1,0.472504,0.000,0.000000,0.014151
1,0.618705,1,0.5,0,0.248319,0.000,0.000000,0.025374
2,0.669065,0,0.5,1,0.287881,0.000,0.000000,0.020495
3,0.170264,0,1.0,1,0.274693,0.000,0.000000,0.015412
4,0.952038,1,0.0,0,0.630753,0.125,0.111111,0.154588


In [8]:
titanic = titanic.dropna()
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.683453,0,1.0,1,0.472504,0.000,0.000000,0.014151
1,0.618705,1,0.5,0,0.248319,0.000,0.000000,0.025374
2,0.669065,0,0.5,1,0.287881,0.000,0.000000,0.020495
3,0.170264,0,1.0,1,0.274693,0.000,0.000000,0.015412
4,0.952038,1,0.0,0,0.630753,0.125,0.111111,0.154588


In [9]:
trainingSet, validatingSet = DataProcessing.split(titanic,0.7)

In [10]:
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.683453,0,1.0,1,0.472504,0.000,0.000000,0.014151
1,0.618705,1,0.5,0,0.248319,0.000,0.000000,0.025374
2,0.669065,0,0.5,1,0.287881,0.000,0.000000,0.020495
3,0.170264,0,1.0,1,0.274693,0.000,0.000000,0.015412
4,0.952038,1,0.0,0,0.630753,0.125,0.111111,0.154588
